In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#%% import

from numpy.lib.financial import _irr_dispatcher
from pandas.core.algorithms import mode, value_counts
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import os
import pandas as pd
import numpy as np
# import matplotlib as plt
# import seaborn as sns
from sklearn import ensemble
from sklearn import tree
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
#%% read files

# /kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
# /kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
# /kaggle/input/house-prices-advanced-regression-techniques/train.csv
# /kaggle/input/house-prices-advanced-regression-techniques/test.csv

if os.path.isdir("/kaggle/input"):
    # we are on Kaggle.
    file_name_train = "/kaggle/input/house-prices-advanced-regression-techniques/train.csv"
    file_name_test = "/kaggle/input/house-prices-advanced-regression-techniques/test.csv"
    file_name_output = "house_prices_submission.csv"
    # import os
    # for dirname, _, filenames in os.walk('/kaggle/input'):
    #     for filename in filenames:
    #         print(os.path.join(dirname, filename)
else:
    # we are not on Kaggle, local machine possibly.
    file_name_train = "train.csv"
    file_name_test = "test.csv"
    file_name_output = "house_prices_submission.csv"

train_data = pd.read_csv(file_name_train)
test_data = pd.read_csv(file_name_test)
id_data = test_data["Id"].to_frame()
df0 = train_data.copy()

In [ ]:
#%% description

dfdesc = train_data.describe().T
dfdesc

In [ ]:
#%% info

train_data.info()

In [ ]:
#%% correlation according to SalesPrice;

train_data.corrwith(train_data["SalePrice"], method="pearson")

In [ ]:
#%% columns to drop

columns_to_drop = ["Id", "Alley", "FireplaceQu",
                   "PoolQC", "Fence", "MiscFeature"]
# Alley: 91 non-null
# FireplaceQu: 770
# PoolQC: 7
# Fence: 281
# MiscFeature: 54
df = train_data.drop(columns_to_drop, axis=1)
df2 = test_data.drop(columns_to_drop, axis=1)

In [ ]:
#%% select categoric ones

dfcategorics = df.select_dtypes(include=['object'])
print(dfcategorics.columns)

In [ ]:
#%% nulls

dfnulls = df2.isnull().sum()

In [ ]:
# %% value_counts

def display_value_counts(df1):
    column_names = """
    BsmtQual         44
    BsmtCond         45
    BsmtFinType1     42
    BsmtFinType2     42
    KitchenQual       1
    GarageFinish     78
    GarageQual       78
    GarageCond       78
    """.strip()

    for line in column_names.splitlines():
        column_name = line.split()[0]
        print()
        print(column_name)
        value_counts = df1[column_name].value_counts()
        print(value_counts)

display_value_counts(df)
display_value_counts(df2)

In [ ]:
# %% fillna

def fillna(df1):

    # filling na values with
    # - a constant value (no)
    # - mean, median or mode of the column (tried)
    # - interpolate function

    # TODO: değerleri korelasyona bakarak doldur.
    # it can be also filled with mode
    df1["BsmtQual"] = df1["BsmtQual"].fillna("TA")
    df1["BsmtCond"] = df1["BsmtCond"].fillna("TA")
    df1["BsmtFinType1"] = df1["BsmtFinType1"].fillna("GLQ")
    df1["BsmtFinType2"] = df1["BsmtFinType2"].fillna("Unf")
    df1["KitchenQual"] = df1["KitchenQual"].fillna("TA")
    df1["GarageFinish"] = df1["GarageFinish"].fillna("Unf")
    df1["GarageQual"] = df1["GarageQual"].fillna("TA")
    df1["GarageCond"] = df1["GarageCond"].fillna("TA")

    # method='pad' -> taking values from the previous row.
    df1["LotFrontage"] = df1["LotFrontage"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["MasVnrArea"] = df1["MasVnrArea"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["GarageYrBlt"] = df1["GarageYrBlt"].interpolate(method='linear', direction = 'forward', inplace=True)

    df1["BsmtFinSF1"] = df1["BsmtFinSF1"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["BsmtFinSF2"] = df1["BsmtFinSF2"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["BsmtUnfSF"] = df1["BsmtUnfSF"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["TotalBsmtSF"] = df1["TotalBsmtSF"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["BsmtFullBath"] = df1["BsmtFullBath"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["BsmtHalfBath"] = df1["BsmtHalfBath"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["GarageCars"] = df1["GarageCars"].interpolate(method='linear', direction = 'forward', inplace=True)
    df1["GarageArea"] = df1["GarageArea"].interpolate(method='linear', direction = 'forward', inplace=True)

fillna(df)
fillna(df2)

In [ ]:
# %%
def build_df(df1):
    df = df1.copy()
    dfi = df.select_dtypes(include=['int64'])
    dff = df.select_dtypes(include=['float64'])

    ordinal_columns = """
ExterQual
ExterCond
BsmtQual
BsmtCond
BsmtFinType1
BsmtFinType2
HeatingQC
KitchenQual
GarageFinish
GarageQual
GarageCond
PavedDrive
    """.strip().split("\n")

    dfo = df[ordinal_columns]
    dfx = pd.concat([dfi, dfo, dff], axis=1)
    return dfx

df = build_df(df)
df2 = build_df(df2)

In [ ]:
# %% encode ordinal columns

def encode_ordinal(df0, column_name: str, categories):
    """
    Label Encodes on a custom order.

    df0: DataFrame
    column_name: str
    categories: List of strings

    Can bu used for ordinal mapping.
    It works on a copy of DataFrame df0.
    It overwrites the column in this copy.

    requires:
        from sklearn.preprocessing import LabelEncoder
    categories: ["red", "green", "blue"]
                  0      1        2

    example:
        df["ExterQual2"] = df["ExterQual"]
        df2 = encode_ordinal(df, "ExterQual2", ["Ex", "Gd", "TA", "Fa", "Po"])

    returns a modified copy of the original DataFrame.
    """
    df = df0.copy()
    mapping = {}
    for i, category in enumerate(categories):
        mapping[category] = i

    df[column_name] = df[column_name].map(mapping)
    return df


def encode_ordinal_columns(df0):
    df1 = df0.copy()
    values_qualities = ["Ex", "Gd", "TA", "Fa", "Po"]
    values_qualities_with_na = ["NA", "Ex", "Gd", "TA", "Fa", "Po"]
    values_fintype = ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"]
    values_garagefinish = ["NA", "Unf", "RFn", "Fin"]
    values_paveddrive = ["N", "P", "Y"]

    df1 = encode_ordinal(df1, "ExterQual", values_qualities)
    df1 = encode_ordinal(df1, "ExterCond", values_qualities)
    df1 = encode_ordinal(df1, "BsmtQual", values_qualities_with_na)
    df1 = encode_ordinal(df1, "BsmtCond", values_qualities_with_na)
    df1 = encode_ordinal(df1, "BsmtFinType1", values_fintype)
    df1 = encode_ordinal(df1, "BsmtFinType2", values_fintype)
    df1 = encode_ordinal(df1, "HeatingQC", values_qualities)
    df1 = encode_ordinal(df1, "KitchenQual", values_qualities)
    df1 = encode_ordinal(df1, "GarageFinish", values_garagefinish)
    df1 = encode_ordinal(df1, "GarageQual", values_qualities_with_na)
    df1 = encode_ordinal(df1, "GarageCond", values_qualities_with_na)
    df1 = encode_ordinal(df1, "PavedDrive", values_paveddrive)
    return df1


df = encode_ordinal_columns(df)
df2 = encode_ordinal_columns(df2)

In [ ]:
#%% check null values again

dfnulls1 = pd.isnull(df).sum()
dfnulls2 = pd.isnull(df2).sum()

In [ ]:
#%% train test split

target = "SalePrice"
x = df.drop(target, axis=1)
y = df[target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
                                                    random_state=18)

In [ ]:
#%% Machine Learning

models = [LinearRegression(), RandomForestRegressor(random_state=23),
          GradientBoostingRegressor(n_estimators=500, random_state=23)]

for model in models:
    model.fit(x_train, y_train)
    print("_" * 20)
    print(str(model))
    score = model.score(x_test, y_test)
    print(score)

In [ ]:
#%% extract output

# best model: GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=500, random_state=23)
model.fit(x_train, y_train)

predictions = model.predict(df2)

my_dict = {"Id": id_data["Id"], "SalePrice": predictions}
dfout = pd.DataFrame(my_dict)
dfout.to_csv(file_name_output, index=False)